# Output

In the [former tutorial](./dglap.ipynb) we computed an evolution operator, but we didn't use it
In this one, we'll then explore the content a structure of a computed operator.

In [1]:
import eko

First of all, we need to recompute the operator, since we didn't save it.

In [2]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
evolution_operator = eko.run_dglap(th_card, op_card)

Evolution: computing operators: - 5/5 took: 0.106152 s
Evolution: computing operators: - 4/5 took: 0.125419 s
Evolution: computing operators: - 2/5 took: 0.158712 sEvolution: computing operators: - 3/5 took: 0.158775 s

Evolution: computing operators: - 1/5 took: 0.161731 s
Evolution: computing operators: - 5/5 took: 0.102229 s
Evolution: computing operators: - 4/5 took: 0.121632 s
Evolution: computing operators: - 3/5 took: 0.142097 s
Evolution: computing operators: - 1/5 took: 0.154657 s
Evolution: computing operators: - 2/5 took: 0.160252 s
Evolution: computing operators: - 5/5 took: 0.103914 s
Evolution: computing operators: - 4/5 took: 0.129496 s
Evolution: computing operators: - 3/5 took: 0.150699 s
Evolution: computing operators: - 2/5 took: 0.158504 s
Evolution: computing operators: - 1/5 took: 0.169050 s


Now that we have it, we can actually use one of the available formats to dump it.

In [3]:
evolution_operator.dump_tar("myeko.tar")

Once dumped, we con always use the paired method to load it, at any later time.

In [4]:
myeko = eko.output.Output.load_tar("myeko.tar")
type(myeko)

eko.output.Output

Now, let's inspect the content of the operator.

In [5]:
print(*myeko.keys(), sep="\n")
myeko["eko_version"]

Q2grid
eko_version
inputgrid
inputpids
interpolation_is_log
interpolation_polynomial_degree
interpolation_xgrid
q2_ref
targetgrid
targetpids


'0.0.0'

In the last step, we proved that an `eko.output.Output` object essentially behaves like a dictionary. Indeed, it is a dictionary.

In [6]:
isinstance(myeko, dict)

True

At the moment we have only seen one attribute in action: an `Output` object records the version of `eko` that has generated.
Let's have a look at some other ones.

In [7]:
interpl = myeko["interpolation_is_log"]
interpd = myeko["interpolation_polynomial_degree"]
interpg = myeko["interpolation_xgrid"]
print(f"INTERNAL\nlog? {interpl}, degree? {interpd}\n{interpg}\n")

ig = myeko["inputgrid"]
ip = myeko["inputpids"]
q0 = myeko["q2_ref"]
print(f"INPUT\ngrid: {ig.shape} {ig.dtype}\npids: {len(ip)} {type(ip[0])}\nQ0^2: {q0} GeV^2")
tg = myeko["targetgrid"]
tp = myeko["targetpids"]
print(f"TARGET\ngrid: {tg.shape} {tg.dtype}\npids: {len(tp)} {type(tp[0])}")

print("\nAll grid the same?", (interpg == ig).all() and (interpg == tg).all())

INTERNAL
log? True, degree? 4
[0.001 0.01  0.1   0.5   1.   ]

INPUT
grid: (5,) float64
pids: 14 <class 'int'>
Q0^2: 1.0 GeV^2
TARGET
grid: (5,) float64
pids: 14 <class 'int'>

All grid the same? True


So an `Output` object has some internal parameters, related to the interpolation used for the calculation, and then some external attributes, related to the final operator delivered.
But actually, we have not accessed yet the actual operator.

In [8]:
opgrid = myeko["Q2grid"]
opgrid.keys()

dict_keys([100.0])

Even the operator grid is a dictionary, mapping $Q^2$ values to the operator evolving to that scale (from the unique starting scale $Q_0^2$).
In the present case there is a unique final scale, but in the general case there might be many.

In [9]:
print(opgrid[100.].keys())
op = opgrid[100.]["operators"]
operr = opgrid[100.]["operator_errors"]
alphas = opgrid[100.]["alphas"]

print(f"\nOPERATOR\n{op.shape} {op.dtype}")
print(f"\nERROR\n{operr.shape} {operr.dtype}")
print(f"\nALPHA_S: {alphas}")

dict_keys(['operators', 'alphas', 'operator_errors'])

OPERATOR
(14, 5, 14, 5) float64

ERROR
(14, 5, 14, 5) float64

ALPHA_S: 0.1730886068066815
